### Exercice 2: _Extraction de Données en Ligne_
**Objectif:** Ecrivons les scripts d'extraction de données depuis différentes plateformes.

In [15]:
from urllib.request import urlopen
from requests import get
from requests import request
from bs4 import BeautifulSoup
from dotenv import dotenv_values

1. **Amazon:** Web scraping avec BeautifulSoup: Produits, prix, avis

In [8]:
url = "https://www.amazon.com/dp/B0CP22DQQS?th=1"
html = urlopen(url)
soup = BeautifulSoup(html, 'html.parser')

In [9]:
title = soup.find(id="productTitle").get_text(strip=True)
price = soup.find('span', {'class': 'a-price'}).get_text(strip=True).split('$')[1]
rating = soup.find(class_="a-icon-alt").get_text()
print(f"Produit: {title}\nPrix: {price}$\nAvis: {rating}")

Produit: Marsail Ergonomic Office Chair: Office Computer Desk Chair with High Back Mesh and Adjustable Lumbar Support Rolling Work Swivel Task Chairs with Wheel 3D Armrests and Headrest
Prix: 105.58$
Avis: 4.4 out of 5 stars


 2. **Twitter:** Utilisation de l'API Twitter v2: Tweets, likes, retweets

In [20]:
x_url="https://api.twitter.com/2"

config = dotenv_values(".env")

X_BEARER_TOKEN = config.get("X_BEARER_TOKEN")

headers = {
  'Authorization': f"Bearer {X_BEARER_TOKEN}"
}

if not X_BEARER_TOKEN:
   raise Exception("Bearer token non trouvé. Merci de fournir un valeur à la variable d'environnement X_BEARER_TOKEN.")

def get_tweets(username: str):
    response = get(f"{x_url}/tweets/search/recent?query=from:{username}", headers=headers)
    if response.status_code != 200:
        raise Exception(f"Erreur {response.status_code}: {response.text}")
    data = response.json()
    print(data)

get_tweets("DylanCalluy")

{'title': 'Too Many Requests', 'detail': 'Too Many Requests', 'type': 'about:blank', 'status': 429}
